In [ ]:
"""
문제
mnist 데이터에 대하여 cnn deeplearning 모델을 구성하였다. 이 모델에 대하여 learning-rate와 optimizer에 대하여 
parameter tuning을 하고자 한다. 이를 구현하시오
- 평가자 리스트
- gridsearchcv를 이해하고 있는가
- gridsearchcv를 keras model에 적용할수 있는가
- gridsearchcv의 매개변수 종류를 이해하고 있는가
- pipeline과 연결해서 작업의 편리를 도모할수 있는가
- optimization의 대한 파라미터 대상을 알고있는가
""" 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics

# 사이킷, 케라스 같이쓰기 : 사이킷의 파이프라인, 평가함수 쓰기
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical #원핫인코딩
from tensorflow.keras.callbacks import EarlyStopping


(x_tr, y_tr), (x_te, y_te) = mnist.load_data()
x_tr = x_tr.reshape(x_tr.shape[0], 28, 28, 1) / 255.
x_te = x_te.reshape(x_te.shape[0], 28, 28, 1) / 255.
y_tr = to_categorical(y_tr)
y_te = to_categorical(y_te)

def test_model(optimizer='Adam', lr='0.0002'):
    import tensorflow as tf
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu')) 
    # 필터의 개수 28 - 3 + 1 =26 => ?x26x26x32

    model.add(Conv2D(64, (3, 3), activation='relu')) # ?x24x24x64
    model.add(MaxPooling2D(pool_size=2)) #?x17x17x64
    model.add(Dropout(0.25)) #과적합 방지를 위해서 계산 회로를 임의적 줄임
    model.add(Flatten()) # ?x(17x17x64)

    # FFNN파트
    model.add(Dense(128,  activation='relu')) # ?x(17x17x64) * 128 => ?x128
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax')) # ?x128 => 128x10 => ?x10

    #모델을 컴파일 합니다. 

    opt = eval('tf.keras.optimizers.{}(lr={})'.format(optimizer, lr))


    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,
                  metrics=['accuracy'])    
    return model

seed = 7
np.random.seed(seed)

from sklearn.model_selection import GridSearchCV


optimizer = ['RMSprop', 'Adam', 'Nadam']
lr = ['0.0002', '0.002']
param_grid = dict(optimizer=optimizer, lr=lr) #baseline_mode에 파라메터 할당
grid = GridSearchCV(KerasClassifier(build_fn= test_model, epochs=1, batch_size=200, verbose=1), param_grid=param_grid, n_jobs=-1, verbose=0)
grid_result = grid.fit(x_tr, y_tr)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

C:\Users\2-18\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
